### Libraries

In [23]:
import pandas as pd
import numpy as np
import time
from datetime import date, datetime, timedelta

from tqdm.notebook import tqdm

import tensorflow_hub as hub
from scipy.spatial.distance import cdist

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

##### Days

In [24]:
days=pd.read_csv("days.csv")
days=days.drop(columns="Unnamed: 0")

days = days[days["description"].notnull()]
days

,Date,Name,Country,Year,Month,Day,description,image
0,0009-09-01,Battle of the Teutoburg Forest,NaN,9,9,1,"Allied Germanic peoples, possibly including t...",http://commons.wikimedia.org/wiki/Special:File...
1,0015-01-01,Battle of the Bridges,NaN,15,1,1,Coalition intervention Naval operations Air c...,http://commons.wikimedia.org/wiki/Special:File...
2,0016-01-01,Battle of the Weser River,NaN,16,1,1,Gallic Wars (58 BC – 57 BC) Clades Lolliana (1...,http://commons.wikimedia.org/wiki/Special:File...
3,0030-04-07,Pilate's court,NaN,30,4,7,"In the canonical gospels, Pilate's court refer...",http://commons.wikimedia.org/wiki/Special:File...
4,0043-01-01,Battle of the Medway,NaN,43,1,1,"The Battle of the Medway took place in 43 AD,...",http://commons.wikimedia.org/wiki/Special:File...
...,...,...,...,...,...,...,...,...
55977,1955-06-05 00:00:00,Hervé This,NaN,1955,6,5,birthday,NaN
55978,1921-01-01 00:00:00,Charles Le Gai Eaton,NaN,1921,1,1,birthday,NaN
55979,1913-09-19 00:00:00,Frances Farmer,NaN,1913,9,19,birthday,NaN
55980,1936-12-29 00:00:00,Mary Tyler Moore,NaN,1936,12,29,birthday,NaN


##### Books

In [25]:
books=pd.read_csv("best_books.csv")
books=books.drop(columns="Unnamed: 0")

books

,id,title,authorName,rating,description,rating_count,review_count,img_url,url,title_language
0,3,Harry Potter and the Sorcerer's Stone,J.K. Rowling,4.48,Harry Potter s life is miserable His parents...,8379630,132348,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/3.Harry_Po...,en
1,43504,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.48,Harry Potter s life is miserable His parents...,8379055,132336,https://d15be2nos83ntc.cloudfront.net/images/n...,https://www.goodreads.com/book/show/43504.Harr...,en
2,49772,Harri Potter a Maen yr Athronydd,J.K. Rowling,4.48,Rescued from the outrageous neglect of his au...,8378971,132333,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/49772.Harr...,en
3,2767052,The Hunger Games,Suzanne Collins,4.32,Could you survive on your own in the wild w...,7288063,183095,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/2767052-th...,en
4,12024,Twilight,Stephenie Meyer,3.62,About three things I was absolutely positive ...,5667779,113710,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/12024.Twil...,en
...,...,...,...,...,...,...,...,...,...,...
44458,3401778,The Book on Leadership,John F. MacArthur Jr.,4.14,In The Book on Leadership best selling aut...,1000,89,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/3401778-th...,en
44459,153664,The Scavenger's Guide to Haute Cuisine,Steven Rinella,4.35,Describes one man s efforts to live on the la...,1000,99,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/153664.The...,en
44460,421398,China Marine: An Infantryman's Life After Worl...,Eugene B. Sledge,4.25,See E B Sledge s story in the HBO miniserie...,1000,76,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/421398.Chi...,en
44461,38757,The Price,Arthur Miller,3.82,In a building slated for imminent demolition ...,1000,91,https://d15be2nos83ntc.cloudfront.net/images/n...,https://www.goodreads.com/book/show/38757.The_...,en


# Model

##### universal-sentence-encoder ****

###### TMDB API + Goodreads

In [ ]:
#### code takes +- 2 hours ######

movies=pd.read_csv("TMDB_movies_final.csv")
movies = movies.rename(columns={"overview": "description"})

today = datetime.today()
# today = datetime.today() + timedelta(days=11) #--- use if you want to decide the day

days = days[(days["Month"] == today.month) & (days["Day"] == today.day)].reset_index(drop=True)

# Start time
start_time = time.time()

# Model
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to embed text
def embed_text(text):
    if isinstance(text, str) and text.strip():  # Check if the text is not empty
        return model([text]).numpy()
    else:
        return None

# Starting the matches file
matches = days.copy()

# Opening the matches lists
match_movie1 = []
match_movie1_descr = []
match_movie2 = []
match_movie2_descr = []
match_movie3 = []
match_movie3_descr = []
match_book1 = []
match_book1_descr = []
match_book2 = []
match_book2_descr = []
match_book3 = []
match_book3_descr = []

for index, day in matches.iterrows():
    if day["description"] == "birthday":
        matched_books = books[books['authorName'] == day['Name']]

        # lets check if its an author or an actor
        if len(matched_books) >= 3:
            # Randomly select 3 books
            matched_books = matched_books.sample(3)

            # Append the book information to the respective lists
            match_book1.append(matched_books.iloc[0]['title'])
            match_book1_descr.append(matched_books.iloc[0]['description'])
            match_book2.append(matched_books.iloc[1]['title'])
            match_book2_descr.append(matched_books.iloc[1]['description'])
            match_book3.append(matched_books.iloc[2]['title'])
            match_book3_descr.append(matched_books.iloc[2]['description'])
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are matched books but less than 3
        elif len(matched_books) > 0:
            match_book1.append(matched_books.iloc[0]['title'])  # Select just one book
            match_book1_descr.append(matched_books.iloc[0]['description'])  # Select just one book
            match_book2.append(None)  # Append None as there is no second book
            match_book2_descr.append(None)  # Append None as there is no second book
            match_book3.append(None)  # Append None as there is no third book
            match_book3_descr.append(None)  # Append None as there is no third book
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are no matched books
        else:
            match_book1.append(None)  # Append None as there is no book
            match_book1_descr.append(None)  # Append None as there is no book
            match_book2.append(None)  # Append None as there is no book
            match_book2_descr.append(None)  # Append None as there is no book
            match_book3.append(None)  # Append None as there is no book
            match_book3_descr.append(None)  # Append None as there is no book

            # And lets go to actors

            matched_movies = movies[movies['actors'].fillna('').str.contains(day['Name'])]

            if len(matched_movies) >= 3:
                # Randomly select 3 movies
                matched_movies = matched_movies.sample(3)

                # Append the movie information to the respective lists
                match_movie1.append(matched_movies.iloc[0]['title'])
                match_movie1_descr.append(matched_movies.iloc[0]['description'])
                match_movie2.append(matched_movies.iloc[1]['title'])
                match_movie2_descr.append(matched_movies.iloc[1]['description'])
                match_movie3.append(matched_movies.iloc[2]['title'])
                match_movie3_descr.append(matched_movies.iloc[2]['description'])

            # If there are matched movies but less than 3
            elif len(matched_movies) > 0:
                match_movie1.append(matched_movies.iloc[0]['title'])  # Select just one movie
                match_movie1_descr.append(matched_movies.iloc[0]['description'])  # Select just one movie
                match_movie2.append(None)  # Append None as there is no second movie
                match_movie2_descr.append(None)  # Append None as there is no second movie
                match_movie3.append(None)  # Append None as there is no third movie
                match_movie3_descr.append(None)  # Append None as there is no third movie

            # If there are no matched movies
            else:
                match_movie1.append(None)  # Append None as there is no movie
                match_movie1_descr.append(None)  # Append None as there is no movie
                match_movie2.append(None)  # Append None as there is no movie
                match_movie2_descr.append(None)  # Append None as there is no movie
                match_movie3.append(None)  # Append None as there is no movie
                match_movie3_descr.append(None)  # Append None as there is no movie

    else:
        day_text = day['Name'] if pd.isna(day['description']) else day['Name'] + " " + day['description']
        day_vec = embed_text(day_text)

        print("Encoding movie descriptions...")
        movies['vec'] = [embed_text(desc) for desc in tqdm(movies['description'])]
        print("Encoding book descriptions...")
        books['vec'] = [embed_text(desc) for desc in tqdm(books['description'])]

        movies['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                                movies['vec']]
        books['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                               books['vec']]

        similar_movies = movies.sort_values(by='similarity', ascending=False)
        similar_books = books.sort_values(by='similarity', ascending=False)

        match_movie1.append(similar_movies['title'].iloc[0])
        match_movie1_descr.append(similar_movies['description'].iloc[0])
        match_movie2.append(similar_movies['title'].iloc[1])
        match_movie2_descr.append(similar_movies['description'].iloc[1])
        match_movie3.append(similar_movies['title'].iloc[2])
        match_movie3_descr.append(similar_movies['description'].iloc[2])
        match_book1.append(similar_books['title'].iloc[0])
        match_book1_descr.append(similar_books['description'].iloc[0])
        match_book2.append(similar_books['title'].iloc[1])
        match_book2_descr.append(similar_books['description'].iloc[1])
        match_book3.append(similar_books['title'].iloc[2])
        match_book3_descr.append(similar_books['description'].iloc[2])

    print(day['Name'])

matches["match_movie1"] = match_movie1
matches["match_movie1_descr"] = match_movie1_descr
matches["match_movie2"] = match_movie2
matches["match_movie2_descr"] = match_movie2_descr
matches["match_movie3"] = match_movie3
matches["match_movie3_descr"] = match_movie3_descr
matches["match_book1"] = match_book1
matches["match_book1_descr"] = match_book1_descr
matches["match_book2"] = match_book2
matches["match_book2_descr"] = match_book2_descr
matches["match_book3"] = match_book3
matches["match_book3_descr"] = match_book3_descr

matches = matches.dropna(subset=['match_movie1', 'match_book1'], how='all').reset_index(drop=True)
matches.drop_duplicates(subset=['description', 'match_movie1'], inplace=True)

matches.to_csv(f"matches{today.strftime('%d%m%Y')}_TMDB.csv", index=False)

# End time
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

matches

Encoding movie descriptions...


100%|███████████████████████████████████████████████████████████████████████████| 26568/26568 [00:54<00:00, 486.42it/s]


Encoding book descriptions...


100%|███████████████████████████████████████████████████████████████████████████| 44463/44463 [01:36<00:00, 461.42it/s]


Battle of Guadalete
Encoding movie descriptions...


 51%|██████████████████████████████████████▌                                    | 13644/26568 [00:25<00:24, 526.26it/s]

###### IMDB webscrap + Goodreads

In [ ]:
#### code takes +- 2 hours ######

movies=pd.read_csv("imdb_movie_fetch.csv")
movies=movies.drop(columns="Unnamed: 0")

today = datetime.today()
# today = datetime.today() + timedelta(days=10)  # --- use if you want to decide the day

days = days[(days["Month"] == today.month) & (days["Day"] == today.day)].reset_index(drop=True)

# Start time
start_time = time.time()

# Model
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to embed text
def embed_text(text):
    if isinstance(text, str) and text.strip():  # Check if the text is not empty
        return model([text]).numpy()
    else:
        return None

# Starting the matches file
matches2 = days.copy()

# Opening the matches lists
match_movie1 = []
match_movie1_descr = []
match_movie2 = []
match_movie2_descr = []
match_movie3 = []
match_movie3_descr = []
match_book1 = []
match_book1_descr = []
match_book2 = []
match_book2_descr = []
match_book3 = []
match_book3_descr = []

for index, day in matches2.iterrows():
    if day["description"] == "birthday":
        matched_books = books[books['authorName'] == day['Name']]

        # lets check if its an author or an actor
        if len(matched_books) >= 3:
            # Randomly select 3 books
            matched_books = matched_books.sample(3)

            # Append the book information to the respective lists
            match_book1.append(matched_books.iloc[0]['title'])
            match_book1_descr.append(matched_books.iloc[0]['description'])
            match_book2.append(matched_books.iloc[1]['title'])
            match_book2_descr.append(matched_books.iloc[1]['description'])
            match_book3.append(matched_books.iloc[2]['title'])
            match_book3_descr.append(matched_books.iloc[2]['description'])
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are matched books but less than 3
        elif len(matched_books) > 0:
            match_book1.append(matched_books.iloc[0]['title'])  # Select just one book
            match_book1_descr.append(matched_books.iloc[0]['description'])  # Select just one book
            match_book2.append(None)  # Append None as there is no second book
            match_book2_descr.append(None)  # Append None as there is no second book
            match_book3.append(None)  # Append None as there is no third book
            match_book3_descr.append(None)  # Append None as there is no third book
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are no matched books
        else:
            match_book1.append(None)  # Append None as there is no book
            match_book1_descr.append(None)  # Append None as there is no book
            match_book2.append(None)  # Append None as there is no book
            match_book2_descr.append(None)  # Append None as there is no book
            match_book3.append(None)  # Append None as there is no book
            match_book3_descr.append(None)  # Append None as there is no book

            # And lets go to actors

            matched_movies = movies[movies['actors'].fillna('').str.contains(day['Name'])]

            if len(matched_movies) >= 3:
                # Randomly select 3 movies
                matched_movies = matched_movies.sample(3)

                # Append the movie information to the respective lists
                match_movie1.append(matched_movies.iloc[0]['title'])
                match_movie1_descr.append(matched_movies.iloc[0]['description'])
                match_movie2.append(matched_movies.iloc[1]['title'])
                match_movie2_descr.append(matched_movies.iloc[1]['description'])
                match_movie3.append(matched_movies.iloc[2]['title'])
                match_movie3_descr.append(matched_movies.iloc[2]['description'])

            # If there are matched movies but less than 3
            elif len(matched_movies) > 0:
                match_movie1.append(matched_movies.iloc[0]['title'])  # Select just one movie
                match_movie1_descr.append(matched_movies.iloc[0]['description'])  # Select just one movie
                match_movie2.append(None)  # Append None as there is no second movie
                match_movie2_descr.append(None)  # Append None as there is no second movie
                match_movie3.append(None)  # Append None as there is no third movie
                match_movie3_descr.append(None)  # Append None as there is no third movie

            # If there are no matched movies
            else:
                match_movie1.append(None)  # Append None as there is no movie
                match_movie1_descr.append(None)  # Append None as there is no movie
                match_movie2.append(None)  # Append None as there is no movie
                match_movie2_descr.append(None)  # Append None as there is no movie
                match_movie3.append(None)  # Append None as there is no movie
                match_movie3_descr.append(None)  # Append None as there is no movie

    else:
        day_text = day['Name'] if pd.isna(day['description']) else day['Name'] + " " + day['description']
        day_vec = embed_text(day_text)

        print("Encoding movie descriptions...")
        movies['vec'] = [embed_text(desc) for desc in tqdm(movies['description'])]
        print("Encoding book descriptions...")
        books['vec'] = [embed_text(desc) for desc in tqdm(books['description'])]

        movies['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                                movies['vec']]
        books['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                               books['vec']]

        similar_movies = movies.sort_values(by='similarity', ascending=False)
        similar_books = books.sort_values(by='similarity', ascending=False)

        match_movie1.append(similar_movies['title'].iloc[0])
        match_movie1_descr.append(similar_movies['description'].iloc[0])
        match_movie2.append(similar_movies['title'].iloc[1])
        match_movie2_descr.append(similar_movies['description'].iloc[1])
        match_movie3.append(similar_movies['title'].iloc[2])
        match_movie3_descr.append(similar_movies['description'].iloc[2])
        match_book1.append(similar_books['title'].iloc[0])
        match_book1_descr.append(similar_books['description'].iloc[0])
        match_book2.append(similar_books['title'].iloc[1])
        match_book2_descr.append(similar_books['description'].iloc[1])
        match_book3.append(similar_books['title'].iloc[2])
        match_book3_descr.append(similar_books['description'].iloc[2])

    print(day['Name'])

matches2["match_movie1"] = match_movie1
matches2["match_movie1_descr"] = match_movie1_descr
matches2["match_movie2"] = match_movie2
matches2["match_movie2_descr"] = match_movie2_descr
matches2["match_movie3"] = match_movie3
matches2["match_movie3_descr"] = match_movie3_descr
matches2["match_book1"] = match_book1
matches2["match_book1_descr"] = match_book1_descr
matches2["match_book2"] = match_book2
matches2["match_book2_descr"] = match_book2_descr
matches2["match_book3"] = match_book3
matches2["match_book3_descr"] = match_book3_descr

matches2 = matches2.dropna(subset=['match_movie1', 'match_book1'], how='all').reset_index(drop=True)
matches2.drop_duplicates(subset=['description', 'match_movie1'], inplace=True)

matches2.to_csv(f"matches{today.strftime('%d%m%Y')}_IMDB.csv", index=False)

# End time
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

matches2